In [1]:
import sys
 
sys.path.append('..')
from piece_v2 import piece_v2

piece_v2.start(should_send_to_score=False)

In [ ]:
from enum import StrEnum
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
import random
from soundmining_tools.generative import *
import math
from soundmining_tools.modular.instrument import AddAction
from soundmining_tools.sequencer import SequenceNote
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from ipywidgets import Output
from ipycanvas import Canvas, hold_canvas
import ipywidgets as widgets

SOUNDPATH = (
    "/Users/danielstahl/Documents/Music/Pieces/Concrete Music/Concrete Music 14/sounds/Concrete Music 14_v3_sounds"
)

Sounds = StrEnum(
    "Sounds",
    [
        "CARDIOID_HIT_1",
        "OMNI_BIG_RATTLE_1",
        "OMNI_BIG_RATTLE_2",
        "OMNI_HIT_1",
        "OMNI_PAN_HIT_1",
        "OMNI_PAN_HIT_2",
        "OMNI_PAN_SCRATH_1",
        "OMNI_PAN_SCRATH_2",
        "OMNI_SMALL_RATTLE_1",
        "OMNI_SMALL_RATTLE_2",
    ],
)

SoundType = StrEnum("SoundType", ["LOW", "MIDDLE", "HIGH"])

piece_v2.reset()

(
    piece_v2.synth_player
        .add_sound(Sounds.CARDIOID_HIT_1, f"{SOUNDPATH}/Cardioid Hit 1.flac", 0.334, 0.555) # 70, 117, 363, 562, 750, 938, 1137, 1313, 1523, 1723, 1863, 2285, 2425, 2507, 2648, 3105, 3257, 3621, 
        .add_sound(Sounds.OMNI_BIG_RATTLE_1, f"{SOUNDPATH}/Omni Big Rattle 1.flac", 0.0, 2.224)
        .add_sound(Sounds.OMNI_BIG_RATTLE_2, f"{SOUNDPATH}/Omni Big Rattle 2.flac", 0.0, 2.615)
        .add_sound(Sounds.OMNI_HIT_1, f"{SOUNDPATH}/Omni Hit 1.flac", 0.202, 0.450)
        .add_sound(Sounds.OMNI_PAN_HIT_1, f"{SOUNDPATH}/Omni Pan Hit 1.flac", 0.425, 0.997) #94, 293, 434, 574, 703, 891, 1007, 1148, 1301, 1430, 1570, 1723, 1863, 2730, 3293, 3691, 4992, 5062
        .add_sound(Sounds.OMNI_PAN_HIT_2, f"{SOUNDPATH}/Omni Pan Hit 2.flac", 0.413, 0.924)
        .add_sound(Sounds.OMNI_PAN_SCRATH_1, f"{SOUNDPATH}/Omni Pan Scratch 1.flac", 0.096, 1.551)
        .add_sound(Sounds.OMNI_PAN_SCRATH_2, f"{SOUNDPATH}/Omni Pan Scratch 2.flac", 0.043, 1.228)
        .add_sound(Sounds.OMNI_SMALL_RATTLE_1, f"{SOUNDPATH}/Omni Small Rattle 1.flac", 0.062, 2.074)
        .add_sound(Sounds.OMNI_SMALL_RATTLE_2, f"{SOUNDPATH}/Omni Small Rattle 2.flac", 0.015, 2.126)
        .start()
)

SoundGroup = StrEnum("SoundGroup", ["CARDIOID_HIT", "OMNI_BIG_RATTLE", "OMNI_HIT", "OMNI_PAN_HIT", "OMNI_PAN_SCRATCH", "OMNI_SMALL_RATTLE"])

sound_groups = {
    SoundGroup.CARDIOID_HIT: [Sounds.CARDIOID_HIT_1],
    SoundGroup.OMNI_BIG_RATTLE: [Sounds.OMNI_BIG_RATTLE_1, Sounds.OMNI_BIG_RATTLE_2],
    SoundGroup.OMNI_HIT: [Sounds.OMNI_HIT_1], 
    SoundGroup.OMNI_PAN_HIT: [Sounds.OMNI_PAN_HIT_1, Sounds.OMNI_PAN_HIT_2],
    SoundGroup.OMNI_PAN_SCRATCH: [Sounds.OMNI_PAN_SCRATH_1, Sounds.OMNI_PAN_SCRATH_2],
    SoundGroup.OMNI_SMALL_RATTLE: [Sounds.OMNI_SMALL_RATTLE_1, Sounds.OMNI_SMALL_RATTLE_2]
}

static_control = piece_v2.instruments.static_control
sine_control = piece_v2.instruments.sine_control
perc_control = piece_v2.instruments.perc_control
line_control = piece_v2.instruments.line_control
signal_sum = piece_v2.instruments.signal_sum
signal_multiply = piece_v2.instruments.signal_multiply
three_block_control = piece_v2.instruments.three_block_control

sound_group_amp_factor = {
    SoundGroup.CARDIOID_HIT: 1,
    SoundGroup.OMNI_BIG_RATTLE: 2,
    SoundGroup.OMNI_HIT: 1, 
    SoundGroup.OMNI_PAN_HIT: 1,
    SoundGroup.OMNI_PAN_SCRATCH: 2,
    SoundGroup.OMNI_SMALL_RATTLE: 2
}

sound_type_pitch_ranges = {
    SoundType.LOW: (30, 100),
    SoundType.MIDDLE: (200, 700),
    SoundType.HIGH: (1000, 10000)
}

sound_type_amp_factor = {
    SoundType.LOW: 20,
    SoundType.MIDDLE: 10,
    SoundType.HIGH: 10
}

sound_type_pan_points = {
    SoundType.LOW: [(-0.25, 0.25)],
    SoundType.MIDDLE: [(-0.75, -0.25), (0.25, 0.75)],
    SoundType.HIGH:  [(-0.99, -0.75), (0.75, 0.99)]
}


def play_sound(start_time: float, sound_group: SoundGroup, sound_type: SoundType, track: str) -> SequenceNote:
    sound = random.choice(sound_groups[sound_group])

    low_pitch, high_pitch = sound_type_pitch_ranges[sound_type]
    pitch = random_range(low_pitch, high_pitch)
    bw = random_range(100, 300)
    static_amp_factor = sound_type_amp_factor[sound_type] * sound_group_amp_factor[sound_group] * random_range(0.15, 0.85)    
    pan = pan_point(sound_type_pan_points[sound_type])
    rq = bw / pitch
    amp_factor = (1 / math.sqrt(rq)) * static_amp_factor    
    (
        piece_v2.synth_player.note()
        .sound_mono(sound, 1.0, static_control(2.0))
        .mono_band_pass_filter(static_control(pitch), static_control(rq))
        .mono_volume(static_control(amp_factor))
        .pan(static_control(pan))
        .play(start_time=start_time)
    )
    duration = piece_v2.synth_player.get_sound(sound).duration(1.0)
    return SequenceNote(start=start_time, track=track, duration=duration, freq=pitch)

def play_clean_sound(start_time: float, sound_group: SoundGroup):
    sound = random.choice(sound_groups[sound_group])
    static_amp_factor = 2 * sound_group_amp_factor[sound_group] * random_range(0.15, 0.85)
    (
        piece_v2.synth_player.note()
        .sound_mono(sound, 1.0, static_control(static_amp_factor))
        .pan(static_control(random_range(-0.99, 0.99)))
        .play(start_time=start_time)
    )
    
def play_low_hit_pad(start_time: float) -> list[SequenceNote]:
    duration = random_range(3, 5)
    freqs = sorted(random.choices([70, 117, 363, 562], k=2))
    amps = [random_range(0.01, 0.05) for _ in range(2)]
    ring_times = [random_range(0.1, 0.2) for _ in range(2)]
    pan = pan_point(sound_type_pan_points[SoundType.LOW])
    (
        piece_v2.synth_player.note()
            .white_noise(sine_control(0, 1.5))
            .bank_of_resonators(freqs, amps, ring_times)
            .mono_low_pass_filter(static_control(100))
            .mono_high_pass_filter(static_control(30))
            .pan(static_control(pan))
            .play(start_time=start_time, duration=duration)
    )
    track = "LOW_HIT_PAD"
    return [SequenceNote(start=start_time, track=track, duration=duration, freq=freq) for freq in freqs]

def play_high_hit_pad(start_time: float) -> list[SequenceNote]:
    duration = random_range(2, 3)
    freqs = sorted(random.choices([750, 938, 1137, 1313, 1523, 1723, 1863], k=3))
    amps = [random_range(0.05, 0.15) for _ in range(3)]
    ring_times = [random_range(0.01, 0.02) for _ in range(3)]
    pan = pan_point(sound_type_pan_points[SoundType.HIGH])
    (
        piece_v2.synth_player.note()
            .white_noise(sine_control(0, 0.2))
            .bank_of_resonators(freqs, amps, ring_times)
            .mono_low_pass_filter(static_control(10000))
            .mono_high_pass_filter(static_control(1000))
            .pan(static_control(pan))
            .play(start_time=start_time, duration=duration)
    )
    track = "HIGH_HIT_PAD"
    return [SequenceNote(start=start_time, track=track, duration=duration, freq=freq) for freq in freqs]

def play_low_pan_hit_pad(start_time: float) -> list[SequenceNote]:
    duration = random_range(3, 4)
    # (190, 421), 614, 704, 1054, 1148, 1895, 2697. 2719, 2905, 3139, 3680, 4522, 5063
    freqs = sorted(random.choices([190, 421, 614, 704], k=2))
    amps = [random_range(0.05, 0.15) for _ in range(2)]
    ring_times = [random_range(0.03, 0.07) for _ in range(2)]
    pan = pan_point(sound_type_pan_points[SoundType.LOW])
    (
        piece_v2.synth_player.note()
            .white_noise(sine_control(0, 1.0))
            .bank_of_resonators(freqs, amps, ring_times)
            .mono_low_pass_filter(static_control(100))
            .mono_high_pass_filter(static_control(30))
            .pan(static_control(pan))
            .play(start_time=start_time, duration=duration)
    )
    track = "LOW_PAN_HIT_PAD"
    return [SequenceNote(start=start_time, track=track, duration=duration, freq=freq) for freq in freqs]

def play_high_pan_hit_pad(start_time: float) -> list[SequenceNote]:
    duration = random_range(1, 3)
    # (190, 421), 614, 704, 1054, 1148, 1895, 2697. 2719, 2905, 3139, 3680, 4522, 5063
    freqs = sorted(random.choices([1054, 1148, 1895, 2697, 2719, 2905, 3139, 3680, 4522, 5063], k=3))
    amps = [random_range(0.05, 0.15) for _ in range(3)]
    ring_times = [random_range(0.01, 0.03) for _ in range(3)]
    pan = pan_point(sound_type_pan_points[SoundType.HIGH])
    (
        piece_v2.synth_player.note()
            .white_noise(sine_control(0, 0.2))
            .bank_of_resonators(freqs, amps, ring_times)
            .mono_low_pass_filter(static_control(10000))
            .mono_high_pass_filter(static_control(1000))
            .pan(static_control(pan))
            .play(start_time=start_time, duration=duration)
    )
    track = "HIGH_PAN_HIT_PAD"
    return [SequenceNote(start=start_time, track=track, duration=duration, freq=freq) for freq in freqs]    

low_hit_chain = MarkovChain({
    SoundType.LOW: {SoundType.LOW: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.LOW: 1.0, SoundType.MIDDLE: 0}
}, SoundType.LOW)

high_hit_chain = MarkovChain({
    SoundType.HIGH: {SoundType.HIGH: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.HIGH: 1.0, SoundType.MIDDLE: 0}
}, SoundType.HIGH)

low_pan_hit_chain = MarkovChain({
    SoundType.LOW: {SoundType.LOW: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.LOW: 1.0, SoundType.MIDDLE: 0}
}, SoundType.LOW)

high_pan_hit_chain = MarkovChain({
    SoundType.HIGH: {SoundType.HIGH: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.HIGH: 1.0, SoundType.MIDDLE: 0}
}, SoundType.HIGH)

low_rattle_chain = MarkovChain({
    SoundType.LOW: {SoundType.LOW: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.LOW: 1.0, SoundType.MIDDLE: 0}
}, SoundType.LOW)

high_rattle_chain = MarkovChain({
    SoundType.HIGH: {SoundType.HIGH: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.HIGH: 1.0, SoundType.MIDDLE: 0}
}, SoundType.HIGH)

low_pan_scratch_chain = MarkovChain({
    SoundType.LOW: {SoundType.LOW: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.LOW: 1.0, SoundType.MIDDLE: 0}
}, SoundType.LOW)

high_pan_scratch_chain = MarkovChain({
    SoundType.HIGH: {SoundType.HIGH: 0.7, SoundType.MIDDLE: 0.3},
    SoundType.MIDDLE: {SoundType.HIGH: 1.0, SoundType.MIDDLE: 0}
}, SoundType.HIGH)

def play_hit_pulse(start_time: float, duration: float, sound_type_chain: MarkovChain[SoundType], track: str) -> list[SequenceNote]:
    notes = []
    hit_sound_groups = [SoundGroup.CARDIOID_HIT, SoundGroup.OMNI_HIT]
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        sound_group = random.choice(hit_sound_groups)        
        sound_type = sound_type_chain.next()        
        notes.append(play_sound(time, sound_group, sound_type, track))
        time += random_range(3, 5)
    return notes

def play_rattle_pulse(start_time: float, duration:float, sound_type_chain: MarkovChain[SoundType], track: str) -> list[SequenceNote]:
    notes = []
    rattle_sound_groups = [SoundGroup.OMNI_BIG_RATTLE, SoundGroup.OMNI_SMALL_RATTLE]
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        sound_group = random.choice(rattle_sound_groups)
        sound_type = sound_type_chain.next()        
        notes.append(play_sound(time, sound_group, sound_type, track))
        time += random_range(5, 8)
    return notes

def play_low_hit_pad_pulse(start_time: float, duration: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        notes += play_low_hit_pad(time)
        time += random_range(8, 13)
    return notes

def play_high_hit_pad_pulse(start_time: float, duration: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        notes += play_high_hit_pad(time)
        time += random_range(13, 21)
    return notes

def play_pan_hit_pulse(start_time: float, duration: float, sound_type_chain: MarkovChain[SoundType], track: str) -> list[SequenceNote]:   
    notes = [] 
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        sound_group = SoundGroup.OMNI_PAN_HIT
        sound_type = sound_type_chain.next()          
        notes.append(play_sound(time, sound_group, sound_type, track))
        time += random_range(4, 7)
    return notes

def play_pan_scratch_pulse(start_time: float, duration:float, sound_type_chain: MarkovChain[SoundType], track: str) -> list[SequenceNote]: 
    notes = []   
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        sound_group = SoundGroup.OMNI_PAN_SCRATCH
        sound_type = sound_type_chain.next()          
        notes.append(play_sound(time, sound_group, sound_type, track))
        time += random_range(7, 11)
    return notes

def play_low_pan_hit_pad_pulse(start_time: float, duration: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        notes += play_low_pan_hit_pad(time)
        time += random_range(11, 18)
    return notes

def play_high_pan_hit_pad_pulse(start_time: float, duration: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    end_time = start_time + duration
    while time < (end_time - 5):
        notes += play_high_pan_hit_pad(time)
        time += random_range(18, 29)
    return notes

# Lucas numbers 1, 3, 4, 7, 11, 18, 29, 47, 76
# Fibonacci 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89

def play_scene1(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_hit_pulse(time, 55, low_hit_chain, "LOW_HIT")
    time += random_range(1, 2)
    notes += play_rattle_pulse(time, 55, low_rattle_chain, "LOW_RATTLE")
    time += random_range(1, 3)
    notes += play_low_hit_pad_pulse(time, 55)
    return notes

def play_scene2(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_low_pan_hit_pad_pulse(time, 47)
    time += random_range(1, 3)
    notes += play_pan_scratch_pulse(time, 47, low_pan_scratch_chain, "LOW_PAN_SCRATCH")
    time += random_range(1, 2)
    notes += play_pan_hit_pulse(time, 47, low_pan_hit_chain, "LOW_PAN_HIT")
    return notes

def play_scene3(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_hit_pulse(time, 34, high_hit_chain, "HIGH_HIT")
    time += random_range(1, 2)
    notes += play_rattle_pulse(time, 34, high_rattle_chain, "HIGH_RATTLE")
    time += random_range(1, 3)
    notes += play_high_hit_pad_pulse(time, 34)
    return notes

def play_scene4(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_high_pan_hit_pad_pulse(time, 29)
    time += random_range(1, 3)
    notes += play_pan_scratch_pulse(time, 29, high_pan_scratch_chain, "HIGH_PAN_SCRATCH")
    time += random_range(1, 2)
    notes += play_pan_hit_pulse(time, 29, high_pan_hit_chain, "HIGH_PAN_HIT")
    return notes


def play_scene5(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_high_pan_hit_pad_pulse(time, 55)
    time += random_range(1, 3)
    notes += play_hit_pulse(time, 55, low_hit_chain, "LOW_HIT")
    time += random_range(1, 3)
    notes += play_low_hit_pad_pulse(time, 55)
    return notes

def play_scene6(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_pan_hit_pulse(time, 47, high_pan_hit_chain, "LOW_PAN_HIT")
    time += random_range(1, 2)
    notes += play_hit_pulse(time, 47, low_hit_chain, "HIGH_HIT")
    time += random_range(1, 2)
    notes += play_high_pan_hit_pad_pulse(time, 47)
    return notes


def play_scene7(start_time: float) -> list[SequenceNote]:
    notes = []
    time = start_time
    notes += play_pan_scratch_pulse(time, 34, low_pan_hit_chain, "LOW_PAN_SCRATCH")
    time += random_range(1, 3)
    notes += play_rattle_pulse(time, 34, high_hit_chain, "HIGH_RATTLE")
    time += random_range(1, 3)
    notes += play_low_hit_pad_pulse(time, 34)
    return notes

def play_piece(start_time: float) -> list[SequenceNote]:
    time = start_time
    notes = []
    notes += play_scene1(time)
    time += 55
    notes += play_scene2(time)
    time += 47
    notes += play_scene3(time)
    time += 34
    notes += play_scene4(time)
    time += 29
    notes += play_scene6(time)
    time += 47
    notes += play_scene5(time)
    time += 55
    notes += play_scene7(time)

    return notes

class PulseHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.octave:
            case 2:            
                match patch_arguments.note:
                    case 0:
                        play_scene1(patch_arguments.start)
                    case 1:
                        play_scene2(patch_arguments.start)
                    case 2:
                        play_scene3(patch_arguments.start)
                    case 3:
                        play_scene4(patch_arguments.start)
                    case 4:
                        play_scene5(patch_arguments.start)
                    case 5:
                        play_scene6(patch_arguments.start)
                    case 6:
                        play_scene7(patch_arguments.start)
            case 3:
                match patch_arguments.note:
                    case 0:
                        play_hit_pulse(patch_arguments.start, 60, low_hit_chain)
                    case 1:
                        play_hit_pulse(patch_arguments.start, 60, high_hit_chain)
                    case 2:
                        play_rattle_pulse(patch_arguments.start, 60, low_rattle_chain)
                    case 3:
                        play_rattle_pulse(patch_arguments.start, 60, high_rattle_chain)
                    case 4:
                        play_pan_hit_pulse(patch_arguments.start, 60, low_pan_hit_chain)
                    case 5:
                        play_pan_hit_pulse(patch_arguments.start, 60, high_pan_hit_chain)
                    case 6:
                        play_pan_scratch_pulse(patch_arguments.start, 60, low_pan_scratch_chain)
                    case 7:
                        play_pan_scratch_pulse(patch_arguments.start, 60, high_pan_scratch_chain)
                    case 8:
                        play_low_hit_pad_pulse(patch_arguments.start, 60)
                    case 9:
                        play_high_hit_pad_pulse(patch_arguments.start, 60)
                    case 10:
                        play_low_pan_hit_pad_pulse(patch_arguments.start, 60)
                    case 11:
                        play_high_pan_hit_pad_pulse(patch_arguments.start, 60)
            case 4:
                play_piece(patch_arguments.start)
    




class SoundHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.note:
            case 0:
                sound_group = SoundGroup.CARDIOID_HIT
            case 1:
                sound_group = SoundGroup.OMNI_BIG_RATTLE
            case 2:
                sound_group = SoundGroup.OMNI_HIT
            case 3:
                sound_group = SoundGroup.OMNI_PAN_HIT
            case 4:
                sound_group = SoundGroup.OMNI_PAN_SCRATCH
            case 5 | _:
                sound_group = SoundGroup.OMNI_SMALL_RATTLE        

        match patch_arguments.octave:
            case 2:            
                sound_type = None
            case 3:                
                sound_type = SoundType.LOW
            case 4:                
                sound_type = SoundType.MIDDLE
            case 5:                
                sound_type = SoundType.HIGH
            
        if sound_type:
            play_sound(patch_arguments.start, sound_group, sound_type)
        else:
            play_clean_sound(patch_arguments.start, sound_group)



class PadHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)

    def handle_note(self, patch_arguments: PatchArguments) -> None: 
        match patch_arguments.note:
            case 0:
                duration = random_range(3, 5)
                freqs = sorted(random.choices([70, 117, 363, 562], k=2))
                amps = [random_range(0.01, 0.05) for _ in range(2)]
                ring_times = [random_range(0.1, 0.2) for _ in range(2)]
                (
                    piece_v2.synth_player.note()
                        .white_noise(sine_control(0, 1.5))
                        .bank_of_resonators(freqs, amps, ring_times)
                        .mono_low_pass_filter(static_control(100))
                        .mono_high_pass_filter(static_control(30))
                        .pan(static_control(random_range(-0.9, 0.9)))
                        .play(start_time=patch_arguments.start, duration=duration)
                )
            case 1:
                freqs = sorted(random.choices([70, 117, 363, 562], k=2))
                amps = [random_range(0.01, 0.05) for _ in range(2)]
                ring_times = [random_range(0.1, 0.2) for _ in range(2)]
                osc = (
                    piece_v2.synth_player.note()
                        .white_noise(sine_control(0, 1.5))
                        .bank_of_resonators(freqs, amps, ring_times)
                        .mono_low_pass_filter(static_control(100))
                        .mono_high_pass_filter(static_control(30))
                        .audio_stack.pop()
                )                
                freq = random.choices([70, 117, 363, 562])
                mod = (
                    piece_v2.synth_player.note()
                        .sine(static_control(freq), static_control(0.5))                        
                        .audio_stack.pop()
                )
                ring = (
                    signal_multiply(osc, mod).add_action(AddAction.TAIL_ACTION)
                )
                (
                    piece_v2.synth_player.note() 
                    .push(ring)
                    .pan(static_control(random_range(-0.9, 0.9)))
                    .play(patch_arguments.start, duration=random_range(8, 13))
                )

my_handler = PulseHandler(piece_v2.supercollider_client)
piece_v2.receiver.set_note_handler(my_handler)

notes = play_piece(0)

ui_piece = UiPieceBuilder().add_notes(notes).build()

piece_duration = ui_piece.get_duration()

piece_stats = {"total": piece_duration, "total minutes": piece_duration / 60.0, "tracks": len(ui_piece.tracks)}

min_freq = 0
max_freq = 0

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
        min_freq = min(min_freq, note.freq)
        max_freq = max(max_freq, note.freq)

    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 100
NOTE_SCALE_FACTOR = 4
HEIGHT_INDENT = 80

ui_width = 200 + (piece_duration * NOTE_SCALE_FACTOR)
ui_height = TRACK_HEIGHT * len(ui_piece.tracks)

canvas = Canvas(width=ui_width, height=ui_height)

out = Output()

@out.capture()
def handle_mouse_down(x, y):
    canvas.flush()
    print("Mouse down event:", x, y)


canvas.on_mouse_down(handle_mouse_down)
canvas.global_alpha = 0.7

display(canvas)

with hold_canvas():
    for track_index, track in enumerate(sorted(ui_piece.tracks, key=lambda tr: tr.track_name)):
        canvas.font = "14px sans-serif"
        canvas.fill_style = "Black"
        canvas.fill_text(
            track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT
        )
        canvas.stroke_style = "Black"
        canvas.stroke_lines(
            [
                (150, (track_index * TRACK_HEIGHT) + 10),
                (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10)),
            ]
        )
        for note in track.notes:
            relative_note = (note.freq - min_freq) / (max_freq - min_freq)
            startx = 200 + (note.start * NOTE_SCALE_FACTOR)
            starty = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR
            peaky = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
                - 5
            )
            endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
            endy = (
                (track_index * TRACK_HEIGHT)
                - (relative_note * HEIGHT_INDENT)
                + HEIGHT_INDENT
            )
            canvas.stroke_style = note.color
            canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])

stop_button = widgets.Button(description="Stop")
status = widgets.Output()
display(stop_button, status)
with status:
    print("Playing")

def stop_playback(b):
    piece_v2.reset()
    canvas.clear()
    status.clear_output()
    with status:
        print("Playback stopped")


stop_button.on_click(stop_playback)


{'total': 298.9030536591781,
 'total minutes': 4.981717560986302,
 'tracks': 12,
 'LOW_HIT': 264.6589100274821,
 'LOW_RATTLE': 49.90411433647937,
 'LOW_HIT_PAD': 296.91846069038104,
 'LOW_PAN_HIT_PAD': 86.02022992833433,
 'LOW_PAN_SCRATCH': 296.8453202962171,
 'LOW_PAN_HIT': 205.48818837718397,
 'HIGH_HIT': 208.49187416933265,
 'HIGH_RATTLE': 298.9030536591781,
 'HIGH_HIT_PAD': 129.0625686341121,
 'HIGH_PAN_HIT_PAD': 253.72571678992534,
 'HIGH_PAN_SCRATCH': 159.80483539622625,
 'HIGH_PAN_HIT': 160.99641282844593}

Canvas(height=1200, width=1395)

Button(description='Stop', style=ButtonStyle())

Output()

In [9]:
piece_v2.stop()